# 01 Freiburg

Hier wird das PDF 'Listen/Freiburg.pdf' lesbar gemacht. Vorgängig habe ich das PDF Freiburg mit Word geöffnet, in jede farbig markierte Zeile ein 'x' reingeschrieben, und das File wieder als PDF gespeichert. 
* Das Skript hier öffnet das PDF 'Listen/Freiburg_word.pdf', macht jede Seite zu einem DF und hängt diese zusammen. 
* es wandlet NaN zu 0 um und 'x' zu 1. 
* es macht eine neue Spalte, welche Kürzel und Leistungsgruppen trennt. 
    -> Das benötige ich um mit Excel die Leistungsgruppen der Kantone zu vergleichen und manuell zu harmonisieren. 

In [1]:
# Sachen importieren
import pandas as pd
import re
import numpy as np
import camelot.io as camelot
import tabula

### PDF-Import

In [2]:
# Word to PDF 
path_word = "Listen/Freiburg_word.pdf"
tables_fr = camelot.read_pdf(path_word, pages = 'all', flavor='lattice')


# alle Seiten zusammenhängen

df = pd.DataFrame()
headers = ['Leistungsbereiche', 'Kürzel', 'Bezeichnung', 'HFR', 'Daler', 
           'Clinique Générale', 'HIB', 'Petit Prince', 'HNE', 'Klinik Bethesda', 
           'CHUV', 'Inselspital', 'HUG']

for page in range(8):
    temp = tables_fr[page].df
    temp = temp[1:]
    temp.columns = headers
    frames = [df, temp]
    df = pd.concat(frames)

df = df.reset_index()
del df['index']


In [3]:
df.head(2)

,Leistungsbereiche,Kürzel,Bezeichnung,HFR,Daler,Clinique Générale,HIB,Petit Prince,HNE,Klinik Bethesda,CHUV,Inselspital,HUG
0,Basispaket,BP \nBasispaket Chirurgie und Innere Medizin,,x,x,,x,,,,,,
1,Programmiertes \nBasispaket,BPE \nBasispaket für elektive Leistungserbringer,,x,x,x,x,,,,,,


In [4]:
# Exportieren
df.to_csv('Exports/freiburg_x.csv', index=False)

Einige wenige x-Werte wurden nicht der richtigen Zeile zugeordnet. Ich gleiche sie manuell mit dem originalen PDF ab und lese das PDF 'Exports/freiburg_xx.csv' wieder ein.

## Datenbereinigung

In [5]:
# csv laden

df = pd.read_csv('Exports/freiburg_xx.csv')
df.head(2)

,Leistungsbereiche,Kürzel,Bezeichnung,HFR,Daler,Clinique Générale,HIB,Petit Prince,HNE,Klinik Bethesda,CHUV,Inselspital,HUG
0,Basispaket,BP \nBasispaket Chirurgie und Innere Medizin,NaN,x,x,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN
1,Programmiertes \nBasispaket,BPE \nBasispaket für elektive Leistungserbringer,NaN,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN


### Werte mit Funktion umwandeln

In [6]:
# Umwandelm mit Funktion
# <- wenn eine Zelle nicht x oder '' entspricht wird sie von der Funktion nicht angetastet, 
# so dass ich allfällige Werte die den Suchbegriffen nicht entsprechen nicht überschreibe und die
# Funktion wenn nötig anpassen kann. 

def search_x(cell):
    if cell == 'x' or cell == 'X':
        return 1
    elif pd.isnull(cell):
        return 0
    else:
        return cell

In [7]:
# Die Liste mit den headers stammt von oben

for column in headers:
    df[column] = df[column].apply(search_x)

In [8]:
# Die Spalte Bezeichnung benötige ich vorläufig nicht
df = df.drop('Bezeichnung', axis = 1)

df.head()

,Leistungsbereiche,Kürzel,HFR,Daler,Clinique Générale,HIB,Petit Prince,HNE,Klinik Bethesda,CHUV,Inselspital,HUG
0,Basispaket,BP \nBasispaket Chirurgie und Innere Medizin,1,1,0,1,0,0,0,0,0,0
1,Programmiertes \nBasispaket,BPE \nBasispaket für elektive Leistungserbringer,1,1,1,1,0,0,0,0,0,0
2,Dermatologie,DER1 \nDermatologie (inkl. Geschlechtskrankhe...,1,1,0,1,0,0,0,0,0,0
3,0,DER1.1 \nDermatologische Onkologie,1,1,0,0,0,0,0,1,1,1
4,0,DER1.2 \nSchwere Hauterkrankungen,1,0,0,0,0,0,0,0,0,0


In [9]:
# Prüfen ob alle Zellen erfasst wurden 
df.dtypes

Leistungsbereiche    object
Kürzel               object
HFR                   int64
Daler                 int64
Clinique Générale     int64
HIB                   int64
Petit Prince          int64
HNE                   int64
Klinik Bethesda       int64
CHUV                  int64
Inselspital           int64
HUG                   int64
dtype: object

In [10]:
#df.info()

### Leistungsgruppe extrahieren

Die Spalte Leistungsgruppe benötige ich um mit Excel anschliessend die Leistungsgruppen der Kantone zu verlgeichen. 

In [11]:
df_leistungsgruppe = df.copy()

# weil die Spalte 'Bezeichnung' gelöscht ist, entferne ich das Element aus meiner headers-Liste
headers.remove('Bezeichnung')
#Spalte nach Zeilenumbruch teilen
searchfor = r'(.*)(\n)(.*)' # in schön: (r'(.*)(\n)(.*)'

df_leistungsgruppe[['Kürzel_2','Umbruch','Leistungsgruppe']] = df_leistungsgruppe['Kürzel'].str.extract(searchfor, expand=True)
df_leistungsgruppe = df_leistungsgruppe.drop('Umbruch', axis=1)
df_leistungsgruppe = df_leistungsgruppe[headers]
df_leistungsgruppe.head(16)

,Leistungsbereiche,Kürzel,HFR,Daler,Clinique Générale,HIB,Petit Prince,HNE,Klinik Bethesda,CHUV,Inselspital,HUG
0,Basispaket,BP \nBasispaket Chirurgie und Innere Medizin,1,1,0,1,0,0,0,0,0,0
1,Programmiertes \nBasispaket,BPE \nBasispaket für elektive Leistungserbringer,1,1,1,1,0,0,0,0,0,0
2,Dermatologie,DER1 \nDermatologie (inkl. Geschlechtskrankhe...,1,1,0,1,0,0,0,0,0,0
3,0,DER1.1 \nDermatologische Onkologie,1,1,0,0,0,0,0,1,1,1
4,0,DER1.2 \nSchwere Hauterkrankungen,1,0,0,0,0,0,0,0,0,0
5,0,DER2 \nWundpatienten,1,0,1,1,0,0,0,0,0,0
6,Hals-Nasen-Ohren,HNO1 \nHals-Nasen-Ohren (HNO-Chirurgie),1,1,1,1,0,0,0,0,0,0
7,0,HNO1.1 \nHals- und Gesichtschirurgie,1,1,1,1,0,0,0,0,0,0
8,0,HNO1.1.1 Komplexe Halseingriffe (Interdiszip...,1,0,0,0,0,0,0,1,1,0
9,0,HNO1.2 \nErweiterte Nasenchirurgie mit Nebenh...,1,1,1,1,0,0,0,0,0,0


<- Das funktioniert, aber bei den Zellen ohne Zeilenumbruch steht nichts, deshalb lasse ich die Spalte 'Kürzel' vorläufig stehen

In [12]:
# Dataframe exportieren

df_leistungsgruppe.to_csv('exports/Freiburg_0.csv', index=False)